In [2]:
# Initial imports.
import pandas as pd
import numpy as np
import datetime as dt
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [3]:
# Loading data after database join of seasons/quarters file and Apple's stock file
# https://towardsdatascience.com/4-tricks-you-should-know-to-parse-date-columns-with-pandas-read-csv-27355bb2ad0e#:~:text=By%20default%2C%20date%20columns%20are%20parsed%20using%20the,a%20different%20date%20format%2C%20for%20example%2C%20YYYY-DD-MM%20HH%3AMM%3ASS%3A
file_path =Path("../AAPL_Pre_Official_DB_7_31.csv")
apple_df = pd.read_csv(file_path, parse_dates=['Date'])
apple_df.head()

Date  Year  Month  Season Quarter      Open      High       Low  \
0 1980-12-12  1980     12  Winter      Q4  0.128348  0.128906  0.128348   
1 1980-12-15  1980     12  Winter      Q4  0.122210  0.122210  0.121652   
2 1980-12-16  1980     12  Winter      Q4  0.113281  0.113281  0.112723   
3 1980-12-17  1980     12  Winter      Q4  0.115513  0.116071  0.115513   
4 1980-12-18  1980     12  Winter      Q4  0.118862  0.119420  0.118862   

      Close  Adj Close  ...  Open_Change  High_Change  Low_Change  \
0  0.128348   0.100751  ...     0.128348     0.128906    0.128348   
1  0.121652   0.095495  ...    -0.006138    -0.006696   -0.006696   
2  0.112723   0.088485  ...    -0.008929    -0.008929   -0.008929   
3  0.115513   0.090676  ...     0.002232     0.002790    0.002790   
4  0.118862   0.093304  ...     0.003349     0.003349    0.003349   

   Close_Change  Volume_Change  Gain_Loss_Open Gain_Loss_High Gain_Loss_Low  \
0      0.128348              0            Gain           Gain          Gain   
1     -0.006696     -293148800            Loss           Loss          Loss   
2     -0.008929      -70156800            Loss           Loss          Loss   
3      0.002790      -19286400            Gain           Gain          Gain   
4      0.003349      -12992000            Gain           Gain          Gain   

  Gain_Loss_Close Gain_Loss_Volume  
0            Gain             Gain  
1            Loss             Loss  
2            Loss             Loss  
3            Gain             Loss  
4            Gain             Loss  

[5 rows x 21 columns]

In [4]:
apple_adj_df = apple_df.drop(['Open_Change','Low_Change','Close_Change','Gain_Loss_Open','Gain_Loss_Low','Gain_Loss_Close','Gain_Loss_Volume'], axis=1)
apple_adj_df.head()

Date  Year  Month  Season Quarter      Open      High       Low  \
0 1980-12-12  1980     12  Winter      Q4  0.128348  0.128906  0.128348   
1 1980-12-15  1980     12  Winter      Q4  0.122210  0.122210  0.121652   
2 1980-12-16  1980     12  Winter      Q4  0.113281  0.113281  0.112723   
3 1980-12-17  1980     12  Winter      Q4  0.115513  0.116071  0.115513   
4 1980-12-18  1980     12  Winter      Q4  0.118862  0.119420  0.118862   

      Close  Adj Close     Volume  High_Change  Volume_Change Gain_Loss_High  
0  0.128348   0.100751  469033600     0.128906              0           Gain  
1  0.121652   0.095495  175884800    -0.006696     -293148800           Loss  
2  0.112723   0.088485  105728000    -0.008929      -70156800           Loss  
3  0.115513   0.090676   86441600     0.002790      -19286400           Gain  
4  0.118862   0.093304   73449600     0.003349      -12992000           Gain

In [5]:
apple_adj_df['Date'] = pd.to_datetime(apple_adj_df['Date'])
apple_adj_df['Date'] = apple_adj_df['Date'].map(dt.datetime.toordinal)
apple_adj_df.head()

Date  Year  Month  Season Quarter      Open      High       Low  \
0  723161  1980     12  Winter      Q4  0.128348  0.128906  0.128348   
1  723164  1980     12  Winter      Q4  0.122210  0.122210  0.121652   
2  723165  1980     12  Winter      Q4  0.113281  0.113281  0.112723   
3  723166  1980     12  Winter      Q4  0.115513  0.116071  0.115513   
4  723167  1980     12  Winter      Q4  0.118862  0.119420  0.118862   

      Close  Adj Close     Volume  High_Change  Volume_Change Gain_Loss_High  
0  0.128348   0.100751  469033600     0.128906              0           Gain  
1  0.121652   0.095495  175884800    -0.006696     -293148800           Loss  
2  0.112723   0.088485  105728000    -0.008929      -70156800           Loss  
3  0.115513   0.090676   86441600     0.002790      -19286400           Gain  
4  0.118862   0.093304   73449600     0.003349      -12992000           Gain

In [6]:
# Convert date into an integer since the system was not taking it as a float so that the ML model can read the date - for now.
# https://www.geeksforgeeks.org/convert-floats-to-integers-in-a-pandas-dataframe/
#apple_df['Date'] = apple_df['Date'].astype(int)

In [8]:
# Must remain in order to activate label_binarize
#If in doubt refer to sklearn
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
apple_gain_loss_df = apple_adj_df.copy()
apple_gain_loss_df['Gain_Loss_High'] = le.fit_transform(apple_gain_loss_df['Gain_Loss_High'])
apple_gain_loss_df.head()

Date  Year  Month  Season Quarter      Open      High       Low  \
0  723161  1980     12  Winter      Q4  0.128348  0.128906  0.128348   
1  723164  1980     12  Winter      Q4  0.122210  0.122210  0.121652   
2  723165  1980     12  Winter      Q4  0.113281  0.113281  0.112723   
3  723166  1980     12  Winter      Q4  0.115513  0.116071  0.115513   
4  723167  1980     12  Winter      Q4  0.118862  0.119420  0.118862   

      Close  Adj Close     Volume  High_Change  Volume_Change  Gain_Loss_High  
0  0.128348   0.100751  469033600     0.128906              0               0  
1  0.121652   0.095495  175884800    -0.006696     -293148800               1  
2  0.112723   0.088485  105728000    -0.008929      -70156800               1  
3  0.115513   0.090676   86441600     0.002790      -19286400               0  
4  0.118862   0.093304   73449600     0.003349      -12992000               0

In [10]:
# If in doubt look up scikit learn label_binarize
from sklearn.preprocessing import label_binarize

binarized_gain_loss = label_binarize(y=list(apple_gain_loss_df['Gain_Loss_High']), classes=[1,0])
apple_gain_loss_df = apple_adj_df.copy()
apple_gain_loss_df['Gain_Loss_High'] = binarized_gain_loss
apple_gain_loss_df.head()

Date  Year  Month  Season Quarter      Open      High       Low  \
0  723161  1980     12  Winter      Q4  0.128348  0.128906  0.128348   
1  723164  1980     12  Winter      Q4  0.122210  0.122210  0.121652   
2  723165  1980     12  Winter      Q4  0.113281  0.113281  0.112723   
3  723166  1980     12  Winter      Q4  0.115513  0.116071  0.115513   
4  723167  1980     12  Winter      Q4  0.118862  0.119420  0.118862   

      Close  Adj Close     Volume  High_Change  Volume_Change  Gain_Loss_High  
0  0.128348   0.100751  469033600     0.128906              0               1  
1  0.121652   0.095495  175884800    -0.006696     -293148800               0  
2  0.112723   0.088485  105728000    -0.008929      -70156800               0  
3  0.115513   0.090676   86441600     0.002790      -19286400               1  
4  0.118862   0.093304   73449600     0.003349      -12992000               1

In [11]:
# Perform binary encoding of Season and Quarter columns
apple_binary_encoded = pd.get_dummies(apple_gain_loss_df, columns=["Season", "Quarter"])
apple_binary_encoded.head()

Date  Year  Month      Open      High       Low     Close  Adj Close  \
0  723161  1980     12  0.128348  0.128906  0.128348  0.128348   0.100751   
1  723164  1980     12  0.122210  0.122210  0.121652  0.121652   0.095495   
2  723165  1980     12  0.113281  0.113281  0.112723  0.112723   0.088485   
3  723166  1980     12  0.115513  0.116071  0.115513  0.115513   0.090676   
4  723167  1980     12  0.118862  0.119420  0.118862  0.118862   0.093304   

      Volume  High_Change  Volume_Change  Gain_Loss_High  Season_Fall  \
0  469033600     0.128906              0               1            0   
1  175884800    -0.006696     -293148800               0            0   
2  105728000    -0.008929      -70156800               0            0   
3   86441600     0.002790      -19286400               1            0   
4   73449600     0.003349      -12992000               1            0   

   Season_Spring  Season_Summer  Season_Winter  Quarter_Q1  Quarter_Q2  \
0              0              0              1           0           0   
1              0              0              1           0           0   
2              0              0              1           0           0   
3              0              0              1           0           0   
4              0              0              1           0           0   

   Quarter_Q3  Quarter_Q4  
0           0           1  
1           0           1  
2           0           1  
3           0           1  
4           0           1

In [13]:
# Define features set
X = apple_binary_encoded.copy()
X = X.drop(["Gain_Loss_High","High_Change"], axis=1)
X.head()

Date  Year  Month      Open      High       Low     Close  Adj Close  \
0  723161  1980     12  0.128348  0.128906  0.128348  0.128348   0.100751   
1  723164  1980     12  0.122210  0.122210  0.121652  0.121652   0.095495   
2  723165  1980     12  0.113281  0.113281  0.112723  0.112723   0.088485   
3  723166  1980     12  0.115513  0.116071  0.115513  0.115513   0.090676   
4  723167  1980     12  0.118862  0.119420  0.118862  0.118862   0.093304   

      Volume  Volume_Change  Season_Fall  Season_Spring  Season_Summer  \
0  469033600              0            0              0              0   
1  175884800     -293148800            0              0              0   
2  105728000      -70156800            0              0              0   
3   86441600      -19286400            0              0              0   
4   73449600      -12992000            0              0              0   

   Season_Winter  Quarter_Q1  Quarter_Q2  Quarter_Q3  Quarter_Q4  
0              1           0           0           0           1  
1              1           0           0           0           1  
2              1           0           0           0           1  
3              1           0           0           0           1  
4              1           0           0           0           1

In [14]:
# Define the target set.
y = apple_df["Gain_Loss_High"].ravel()
y[:5]

array(['Gain', 'Loss', 'Loss', 'Gain', 'Gain'], dtype=object)

In [15]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [16]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=100, random_state=78) 

In [18]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [19]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [20]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

Predicted 0  Predicted 1
Actual 0          896          451
Actual 1          593          617

In [21]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [22]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


Predicted 0  Predicted 1
Actual 0          896          451
Actual 1          593          617

Accuracy Score : 0.5917090340242471
Classification Report
              precision    recall  f1-score   support

        Gain       0.60      0.67      0.63      1347
        Loss       0.58      0.51      0.54      1210

    accuracy                           0.59      2557
   macro avg       0.59      0.59      0.59      2557
weighted avg       0.59      0.59      0.59      2557



In [23]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.12095212, 0.0351549 , 0.04018598, 0.10081584, 0.09808713,
       0.09528324, 0.09944291, 0.10063275, 0.11998046, 0.12897701,
       0.00777693, 0.0077666 , 0.00828414, 0.00771494, 0.00616262,
       0.00792903, 0.00772862, 0.00712479])

In [24]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.12897701452892074, 'Volume_Change'),
 (0.1209521191645504, 'Date'),
 (0.11998046080592992, 'Volume'),
 (0.10081584103858629, 'Open'),
 (0.10063274963749043, 'Adj Close'),
 (0.09944290652692198, 'Close'),
 (0.09808712572160713, 'High'),
 (0.09528323782815964, 'Low'),
 (0.040185981480124526, 'Month'),
 (0.03515490136328672, 'Year'),
 (0.008284139120349547, 'Season_Summer'),
 (0.007929033105051617, 'Quarter_Q2'),
 (0.00777692748505984, 'Season_Fall'),
 (0.007766604662257011, 'Season_Spring'),
 (0.007728615923491548, 'Quarter_Q3'),
 (0.007714936305525231, 'Season_Winter'),
 (0.007124786436446168, 'Quarter_Q4'),
 (0.006162618866241302, 'Quarter_Q1')]